In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import random
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"



def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).to(device).eval()
    return model, tokenizer


def format_query(user_input):
    return f"### User Question ###\n{user_input}Based on the symptoms I described, what disease do you think I have? Please give the disease name before proceeding with the subsequent analysis.\n\n### Assistant Answer ###\n"


def generate_response(model, tokenizer, query):
    inputs = tokenizer(query, return_tensors="pt").to(device)
    gen_kwargs = {"max_new_tokens": 50, "do_sample": True, "temperature": 0.7, "top_k": 50}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(query, "").strip()


def process_excel_no_rag(file_path, output_path, model_path):
    df = pd.read_excel(file_path)

    
    if "Response before RAG" not in df.columns:
        df["Response before RAG"] = ""
    df["Response before RAG"] = df["Response before RAG"].astype(str)

    model, tokenizer = load_model(model_path)

    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing without RAG"):
        symptom_text = row["Symptoms"]
        query_no_rag = format_query(symptom_text)
        response = generate_response(model, tokenizer, query_no_rag)

        
        df.at[i, "Response before RAG"] = str(response)

        
        if i < 20:
            print(f"--- Record {i+1} ---")
            print(f"Symptoms: {symptom_text}")
            print(f"Generated Response: {response}")
            print("\n" + "="*80 + "\n")
    
    df.to_excel(output_path, index=False)


if __name__ == "__main__":
    input_file = "/root/autodl-tmp/nhs_test.xlsx"
    output_file = "/root/autodl-tmp/nhs_test_no_rag.xlsx"
    model_path = "/root/autodl-tmp/trained_model_MedQA"
    
    process_excel_no_rag(input_file, output_file, model_path)


Processing without RAG:   0%|          | 1/410 [00:03<25:17,  3.71s/it]

--- Record 1 ---
Symptoms: In most cases, this condition causes no noticeable symptoms. However, if it becomes large, some people may develop pain or a pulsating feeling in their abdomen (tummy) or persistent back pain. This condition doesn’t usually pose a serious threat to health, but there is a risk that a larger abnormality could rupture. A rupture can cause massive internal bleeding, which is usually fatal. Around 8 out of 10 people with a rupture either die before they reach the hospital or do not survive surgery. The most common symptom of a rupture is sudden and severe abdominal pain. If you suspect that you or someone else has had a rupture, call emergency services immediately and ask for an ambulance.
Generated Response: Aortic aneurysm

### Step 1: The condition described is characterized by the enlargement of a blood vessel, which can become large and painful, or rupture and cause massive internal bleeding.

### Step 2: Aortic aneurys




Processing without RAG:   0%|          | 2/410 [00:06<22:54,  3.37s/it]

--- Record 2 ---
Symptoms: Symptoms may vary from person to person and can appear gradually or suddenly. Common symptoms include swelling, pain that worsens during or after movement or exercise, stiffness that is more pronounced in the morning or after periods of rest, tenderness when touching the affected area, mild heat, and a loss of movement and strength around the ankle and foot.
Generated Response: Rheumatoid Arthritis

### Step 1: Identify the key symptoms
The symptoms described are swelling, pain that worsens during or after movement or exercise, stiffness that is more pronounced in the morning or after periods of rest, tenderness




Processing without RAG:   1%|          | 3/410 [00:09<21:58,  3.24s/it]

...

Processing without RAG:   5%|▍         | 19/410 [01:00<21:20,  3.28s/it]

--- Record 19 ---
Symptoms: Even after receiving adrenaline, individuals experiencing this condition should be taken to a hospital for observation, typically for 6 to 12 hours, as symptoms may occasionally reappear. During hospitalization, oxygen therapy may be provided to assist with breathing, and intravenous fluids may be administered to help stabilize blood pressure. In addition to adrenaline, other medications such as antihistamines and corticosteroids may be used to alleviate symptoms. Blood tests might also be conducted to confirm the condition. Discharge from the hospital is typically possible once symptoms are under control and unlikely to return quickly, though severe cases may require a longer stay. Patients may be advised to continue taking antihistamines and corticosteroids for 2 to 3 days after leaving the hospital to prevent recurrence.
Generated Response: Anaphylaxis

The following are the key features of anaphylaxis:

*   **Rapid onset** of symptoms (usually within 5-3

Processing without RAG:   5%|▍         | 20/410 [01:02<20:34,  3.16s/it]

--- Record 20 ---
Symptoms: The primary symptom of this condition is chest pain, which may present as a dull ache, a heavy sensation, or a feeling of tightness in the chest. The pain can also spread to the arms, neck, jaw, or back and is often triggered by physical exertion. While this condition itself is not life-threatening, prolonged chest pain could indicate a more serious event, such as a heart attack.
Generated Response: Angina pectoris

### Step 1: Identify the key symptoms of the condition
The primary symptom of this condition is chest pain that can present as a dull ache, a heavy sensation, or a feeling of tightness in the chest.




Processing without RAG: 100%|██████████| 410/410 [20:23<00:00,  2.99s/it]
